# Background

In this notebook we'll train a [Logistic Regression model](https://en.wikipedia.org/wiki/Logistic_regression) to distinguish between spam data (food reviews) and legitimate data (Austen). 

Logistic regression is a standard statistical technique used to model a binary variable. In our case the binary variable we are predicting is 'spam' or 'not spam' (i.e. legitimate).  Logistic regression, when combined with a reasonable feature engineering approach, is often a sensible first choice for a classification problem!

We begin by loading in the feature vectors which we generated in either [the simple summaries feature extraction notebook](03-feature-engineering-summaries.ipynb) or [the TF-IDF feature extraction notebook](03-feature-engineering-tfidf.ipynb). 

In [1]:
import pandas as pd
import os.path

feats = pd.read_parquet(os.path.join("data", "features_summaries.parquet"))

When doing exploratory analysis, it's often a good idea to inspect your data as a sanity check.  In this case, we'll make sure that the feature vectors we generated in the last notebook have the shape we expect!

In [2]:
feats.sample(10)

,index,label,no_punct,number_words,mean_wl,max_wl,min_wl,pc_10_wl,pc_90_wl,upper,stop_words
16033,16033,legitimate,28,173,4.560694,12,1,2.0,8.0,26,91
8384,8384,legitimate,20,101,4.603960,15,1,2.0,9.0,13,51
30112,10112,spam,19,89,4.089888,11,1,2.0,7.0,13,35
39567,19567,spam,8,60,3.766667,11,1,2.0,7.0,8,33
22794,2794,spam,9,87,4.563218,11,1,3.0,7.0,7,40
38242,18242,spam,15,83,4.228916,12,1,2.0,7.0,19,38
29722,9722,spam,14,82,4.548780,10,1,2.0,7.9,10,32
13114,13114,legitimate,15,68,4.235294,12,1,2.0,7.0,9,32
5393,5393,legitimate,11,44,4.727273,10,1,2.0,8.7,8,19
5362,5362,legitimate,3,13,4.538462,8,2,3.0,6.0,2,5


The first 2 columns of the `feats` matrix are the index, and label. The remaining columns are the feature vectors. 

We begin by splitting the data into 2 sets: 

* `train` - a set of feature vectors which will be used to train the model
* `test` - a set of feature vectors which will be used to evaluate the model we trained

In [3]:
from sklearn import model_selection
train, test = model_selection.train_test_split(feats, random_state=43)

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
model = LogisticRegression(solver = 'lbfgs', max_iter = 4000)

In [6]:
#training the model
import time

start = time.time()
model.fit(X=train.iloc[:,2:train.shape[1]], y=train["label"])
end = time.time()
print(end - start)


4.023041009902954


With the model trained we can use it to make predictions. We apply the model to the `test` set, then compare the predicted classification of spam or legitimate to the truth.  

In [7]:
predictions = model.predict(test.iloc[:,2:test.shape[1]])

In [8]:
predictions

array(['legitimate', 'spam', 'legitimate', ..., 'spam', 'legitimate',
       'spam'], dtype=object)

We use a binary confusion matrix to visualise the accuracy of the model. 

In [9]:
from mlworkflows import plot

In [10]:
df, chart = plot.binary_confusion_matrix(test["label"], predictions)

In [11]:
chart

alt.Chart(...)

We can look at the raw numbers, and proportions of correctly and incorrectly classified items: 

In [12]:
df

,predicted,actual,raw_count,value
0,legitimate,legitimate,4143,0.827938
1,legitimate,spam,861,0.172062
2,spam,legitimate,852,0.170536
3,spam,spam,4144,0.829464


We can also look at the precision, recall and f1-score for the model. 

In [13]:
from sklearn.metrics import classification_report
print(classification_report(test.label.values, predictions))

              precision    recall  f1-score   support

  legitimate       0.83      0.83      0.83      5004
        spam       0.83      0.83      0.83      4996

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



We want to save the model so that we can use it outside of this notebook.

In [14]:
model

LogisticRegression(max_iter=4000)

In [15]:
from mlworkflows import util
util.serialize_to(model, "model_summaries_logistic.sav")